## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-10-50-29 +0000,bbc,Labour plans new law to ban fracking permanently,https://www.bbc.com/news/articles/cjedpl9kpv2o...
1,2025-10-01-10-49-22 +0000,bbc,Greggs set to raise prices in response to cost...,https://www.bbc.com/news/articles/c2lx2k2y70qo...
2,2025-10-01-10-48-37 +0000,bbc,Why the US government has shut down and what h...,https://www.bbc.com/news/articles/crrj1znp0pyo...
3,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...
4,2025-10-01-10-44-14 +0000,bbc,Baroness Mone-linked company ordered to pay £1...,https://www.bbc.com/news/articles/c1792rk7ynko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,52
22,shutdown,33
13,government,31
2,new,22
162,deal,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...,185
158,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,153
249,2025-09-30-19-20-21 +0000,nypost,Trump threatens ‘irreversible’ actions if Dems...,https://nypost.com/2025/09/30/us-news/trump-th...,133
231,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,129
229,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,125


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,185,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...
231,77,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...
229,73,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
189,56,2025-09-30-22-37-25 +0000,nypost,Americans’ support for Israel plummets as Gaza...,https://nypost.com/2025/09/30/world-news/ameri...
158,56,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
301,46,2025-09-30-15-50-00 +0000,bbc,Trump peace plan 'ignores interests of Palesti...,https://www.bbc.com/news/articles/cx2j97jldkmo...
199,38,2025-09-30-22-04-22 +0000,nypost,Ford CEO warns there’s not enough blue-collar ...,https://nypost.com/2025/09/30/business/ford-ce...
236,36,2025-09-30-19-59-50 +0000,nypost,"Girl, 15, and two cousins tortured to death in...",https://nypost.com/2025/09/30/world-news/argen...
165,35,2025-10-01-00-06-37 +0000,startribune,Principal: Adult older than 21 used fake paper...,https://www.startribune.com/principal-adult-ol...
214,34,2025-09-30-21-24-47 +0000,nypost,Starbucks abruptly closes dozens of NYC locati...,https://nypost.com/2025/09/30/business/starbuc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
